
# Введение в обработку естественного языка
## Урок 7. Сверточные нейронные сети для анализа текста
### Задание Берем отызывы за лето (из архива с материалами или предыдущего занятия)

In [2]:
!pip install pymorphy2
!pip install -U xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
import pandas as pd
import numpy as np
from string import punctuation
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import re
from sklearn.model_selection import train_test_split
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, Flatten
from keras.losses import categorical_crossentropy, SparseCategoricalCrossentropy
import os
import urllib

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
data = pd.read_excel('/content/отзывы за лето (1).xls')
data

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
...,...,...,...
20654,1,"Ну и шляпа,с роот правами бесполезная прога,ра...",2017-06-01
20655,5,Ок,2017-06-01
20656,4,Доволен,2017-06-01
20657,1,"Песопаснасть, рут ни нужын",2017-06-01


### 1. Учим conv сеть для классификации

In [5]:
max_words = 20000
max_len = 150
num_classes = 5

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [6]:
stopwordslist = stopwords.words("russian")
morpher = MorphAnalyzer()
ptrn = r'[^a-zA-Zа-яА-Я0-9]'

def words_only(text):
    text=str(text)
    return text.lower()   

def remove_punkt(text):
    return re.sub(ptrn, ' ', text)

def to_token(text):
    return nltk.tokenize.word_tokenize(text)

def remove_stopwords(text):
    text_list = [w for w in text if w not in stopwordslist]
    return ' '.join(word for word in text_list)

def morphe_text(text):
    text = [morpher.parse(word)[0].normal_form for word in text.split() if word not in stopwordslist]
    return " ".join(text)

def normalize_text(text):
    text = words_only(text)
    text = remove_punkt(text)
    text = to_token(text)
    text = remove_stopwords(text)
    text = morphe_text(text)
    return text

In [7]:
data['Content_norm'] = data['Content'].apply(normalize_text)

In [8]:
train_corpus = " ".join(data['Content_norm'])
train_tokens = word_tokenize(train_corpus)
train_tokens_filtered = [word for word in train_tokens if word.isalnum()]

In [9]:
dist = FreqDist(train_tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [10]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [11]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [12]:
data_train = np.asarray([text_to_sequence(text, max_len) for text in data['Content_norm']], dtype=np.int32)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(data_train, data.Rating, test_size=0.33, random_state=42)

In [14]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train) 
y_test = le.transform(y_test)
le.classes_

array([1, 2, 3, 4, 5])

### 2. Рассмотреть 2-а варианта сеточек
### 2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/

In [15]:
!pip install --upgrade simple_elmo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
urllib.request.urlretrieve("http://vectors.nlpl.eu/repository/20/195.zip", "195.zip")
!unzip "/content/195.zip"

Archive:  /content/195.zip
replace meta.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: meta.json               
  inflating: model.hdf5              
  inflating: options.json            
  inflating: README                  
  inflating: vocab.txt               


In [23]:
from simple_elmo import ElmoModel
model_elmo = ElmoModel()
model_elmo.load('/content/')

2022-06-23 19:37:33,466 : INFO : Loading model from /content/...
2022-06-23 19:37:33,472 : INFO : We will cache the vocabulary of 100 tokens.


ValueError: ignored

In [24]:
embeddings_index = {}
with open('/content/vocab.txt') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

ValueError: ignored

In [19]:
nltk.download('stopwords')
stop_words = set(stopwords.words("russian"))
sw = stop_words
exclude = set(punctuation)
def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

show = data['Content'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


KeyboardInterrupt: ignored

In [25]:
urllib.request.urlretrieve("http://vectors.nlpl.eu/repository/20/181.zip", "181.zip")

('181.zip', <http.client.HTTPMessage at 0x7f70043213d0>)

In [26]:
!unzip "/content/181.zip"

Archive:  /content/181.zip
replace meta.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: meta.json               
  inflating: model.model             
  inflating: model.model.vectors_ngrams.npy  
  inflating: model.model.vectors.npy  
  inflating: model.model.vectors_vocab.npy  
  inflating: README                  


In [27]:
vectors = np.load('/content/model.model.vectors.npy')
len(vectors)

164996

In [28]:
embedding_dim = 300

embedding_matrix = np.zeros((max_words, embedding_dim))
for word in tokens_filtered_top:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[tokens_filtered_top.index(word)] = embedding_vector

In [33]:
model = Sequential()
model.add(Embedding(input_dim=max_words, weights=[embedding_matrix], output_dim=embedding_dim, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [36]:
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split=0.1)

Train on 12456 samples, validate on 1385 samples
Epoch 1/20
12456/12456 [==============================] - ETA: 0s - loss: 1.5980 - accuracy: 0.6780

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


12456/12456 [==============================] - 32s 3ms/sample - loss: 1.5980 - accuracy: 0.6780 - val_loss: 1.5848 - val_accuracy: 0.7011
Epoch 2/20
12456/12456 [==============================] - 33s 3ms/sample - loss: 1.5733 - accuracy: 0.7030 - val_loss: 1.5607 - val_accuracy: 0.7011
Epoch 3/20
12456/12456 [==============================] - 31s 3ms/sample - loss: 1.5494 - accuracy: 0.7030 - val_loss: 1.5370 - val_accuracy: 0.7011
Epoch 4/20
12456/12456 [==============================] - 31s 3ms/sample - loss: 1.5259 - accuracy: 0.7030 - val_loss: 1.5140 - val_accuracy: 0.7011
Epoch 5/20
12456/12456 [==============================] - 32s 3ms/sample - loss: 1.5031 - accuracy: 0.7030 - val_loss: 1.4917 - val_accuracy: 0.7011
Epoch 6/20
12456/12456 [==============================] - 35s 3ms/sample - loss: 1.4810 - accuracy: 0.7030 - val_loss: 1.4700 - val_accuracy: 0.7011
Epoch 7/20
12456/12456 [==============================] - 31s 3ms/sample - loss: 1.4595 - accuracy: 0.7030 - val_loss

In [37]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])



Test score: 1.2224741568135948
Test accuracy: 0.71252567


### 2. Рассмотреть 2-а варианта сеточек
### 2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

In [38]:
epochs = 5
batch_size = 32

In [39]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [40]:
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

In [41]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split=0.1)

Train on 12456 samples, validate on 1385 samples
Epoch 1/5
12448/12456 [============================>.] - ETA: 0s - loss: 0.7802 - accuracy: 0.7246

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


12456/12456 [==============================] - 30s 2ms/sample - loss: 0.7798 - accuracy: 0.7247 - val_loss: 0.6497 - val_accuracy: 0.7690
Epoch 2/5
12456/12456 [==============================] - 28s 2ms/sample - loss: 0.5693 - accuracy: 0.7970 - val_loss: 0.6477 - val_accuracy: 0.7718
Epoch 3/5
12456/12456 [==============================] - 31s 2ms/sample - loss: 0.4796 - accuracy: 0.8268 - val_loss: 0.6844 - val_accuracy: 0.7682
Epoch 4/5
12456/12456 [==============================] - 31s 2ms/sample - loss: 0.4014 - accuracy: 0.8576 - val_loss: 0.7489 - val_accuracy: 0.7661
Epoch 5/5
12456/12456 [==============================] - 34s 3ms/sample - loss: 0.3341 - accuracy: 0.8901 - val_loss: 0.8292 - val_accuracy: 0.7531


In [42]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])



Test score: 0.775729124314906
Test accuracy: 0.7660604
